# Cross-subject decoding input stimuli 
### Ex: Color stims ([red, red] v. [red, blue] v. [blue, red] v. [blue, blue])

#### Takuya Ito
#### 8/18/20

#### Note (8/18/20): Results will not be 'stochastic' if run with a leave-one-out subject CV (and no resampling in trainset)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import multiprocessing as mp
import scipy.stats as stats
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.sandbox.stats.multicomp as mc
import seaborn as sns
import h5py
import tools as tools
from importlib import reload
import nibabel as nib

sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"


In [3]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



projectdir = '/home/ti61/f_mc1689_1/SRActFlow/'

# Using final partition
networkdef = np.loadtxt(projectdir + 'data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = projectdir + 'data/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(list(xticks.keys()))
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# 0.1 Define functions for input stimuli decodings

# 1.0 Run across subject decoding on Color Stimuli decoding
### 4-way coding for every 4 input stimuli combinations
* red, red
* red, blue
* blue, red
* blue, blue

## 1.1 Load data for "Color" stimuli

In [4]:
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
inputtype = 'colorStim'

if inputtype in ['colorStim','oriStim']:
    rois = np.where((networkdef==networkmappings['vis1']) | (networkdef==networkmappings['vis2']))[0] 
elif inputtype in ['constantStim','pitchStim']:
    rois = np.where(networkdef==networkmappings['aud'])[0]

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = tools.loadInputActivity(subj,inputtype)
    scount += 1

/projects/f_mc1689_1/AnalysisTools/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':


#### Decode different stimulus pairings

In [6]:
nproc = 30
ncvs = 1

distances_baseline_color = tools.conditionDecodings(data_task, rois, motorOutput=False, ncvs=ncvs, nproc=nproc)


#### Compute statistics

In [7]:
inputtype = 'colorStim'
statistics_color = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = distances_baseline_color.shape[1]
    p = stats.binom_test(np.mean(distances_baseline_color[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline_color[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics_color[roicount,0] = np.mean(distances_baseline_color[roicount,:])
    statistics_color[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics_color[:,1])
for roicount in range(len(rois)):
    statistics_color[roicount,1] = qs[roicount]
    statistics_color[roicount,2] = h0[roicount]*statistics_color[roicount,0]
    
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics_color[:,1]<0.05)[0]
print('Number of ROIs significant for', inputtype, 'Stimuli:', sig_ind.shape[0])
print('Accuracies:', statistics_color[sig_ind,0])

#### Map back to surface
# Put all data into a single matrix (since we only run a single classification)
inputStim = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in rois:
    vertex_ind = np.where(glasser2==roi+1)[0]
    inputStim[vertex_ind,0] = statistics_color[roicount,0]
    inputStim[vertex_ind,1] = statistics_color[roicount,1]
    inputStim[vertex_ind,2] = statistics_color[roicount,2]

    roicount += 1

# np.savetxt(projectdir + 'data/results/GroupfMRI/InputStimuliDecoding/InputStimuliRegions_COLOR2.csv', rois[np.where(statistics_color[:,1]<0.05)[0]], delimiter=',')
    
# #### 
# # Write file to csv and run wb_command
# outdir = '/projects3/SRActFlow/data/results/GroupfMRI/InputStimuliDecoding/'
# filename = inputtype + 'Decoding2'
# np.savetxt(outdir + filename + '.csv', inputStim,fmt='%s')
# wb_file = filename + '.dscalar.nii'
# wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
# os.system(wb_command)


Number of ROIs significant for colorStim Stimuli: 7
Accuracies: [0.34114583 0.30989583 0.33333333 0.34375    0.33072917 0.34635417
 0.33072917]


# 2.0 Run across subject decoding on Orientation Stimuli decoding
### 4-way coding for every 4 input stimuli combinations
* vertical, vertical
* vertical, horizontal
* horizontal, vertical
* horizontal, horizontal

## 2.1 Load data for "Orientation" stimuli

In [8]:
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
inputtype = 'oriStim'

if inputtype in ['colorStim','oriStim']:
    rois = np.where((networkdef==networkmappings['vis1']) | (networkdef==networkmappings['vis2']))[0] 
elif inputtype in ['constantStim','pitchStim']:
    rois = np.where(networkdef==networkmappings['aud'])[0]

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = tools.loadInputActivity(subj,inputtype)
    scount += 1

/projects/f_mc1689_1/AnalysisTools/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':


#### Decode different stimulus pairings

In [10]:
nproc = 30
ncvs = 1

distances_baseline_ori = tools.conditionDecodings(data_task, rois, motorOutput=False, ncvs=ncvs, nproc=nproc)


#### Compute statistics

In [11]:
inputtype = 'oriStim'
statistics_ori = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = distances_baseline_ori.shape[1]
    p = stats.binom_test(np.mean(distances_baseline_ori[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline_ori[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics_ori[roicount,0] = np.mean(distances_baseline_ori[roicount,:])
    statistics_ori[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics_ori[:,1])
for roicount in range(len(rois)):
    statistics_ori[roicount,1] = qs[roicount]
    statistics_ori[roicount,2] = h0[roicount]*statistics_ori[roicount,0]
    
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics_ori[:,1]<0.05)[0]
print('Number of ROIs significant for', inputtype, 'Stimuli:', sig_ind.shape[0])
print('Accuracies:', statistics_ori[sig_ind,0])

#### Map back to surface
# Put all data into a single matrix (since we only run a single classification)
inputStim = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in rois:
    vertex_ind = np.where(glasser2==roi+1)[0]
    inputStim[vertex_ind,0] = statistics_ori[roicount,0]
    inputStim[vertex_ind,1] = statistics_ori[roicount,1]
    inputStim[vertex_ind,2] = statistics_ori[roicount,2]

    roicount += 1

# np.savetxt('/projects3/SRActFlow/data/results/GroupfMRI/InputStimuliDecoding/InputStimuliRegions_ORI2.csv', rois[np.where(statistics_ori[:,1]<0.05)[0]], delimiter=',')
    
# #### 
# # Write file to csv and run wb_command
# outdir = '/projects3/SRActFlow/data/results/GroupfMRI/InputStimuliDecoding/'
# filename = inputtype + 'Decoding2'
# np.savetxt(outdir + filename + '.csv', inputStim,fmt='%s')
# wb_file = filename + '.dscalar.nii'
# wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
# os.system(wb_command)


Number of ROIs significant for oriStim Stimuli: 13
Accuracies: [0.54427083 0.47135417 0.51041667 0.48177083 0.328125   0.30729167
 0.31510417 0.30989583 0.5546875  0.56770833 0.52864583 0.421875
 0.3125    ]


# 3.0 Run across subject decoding on Pitch Stimuli decoding
### 4-way coding for every 4 input stimuli combinations
* high, high
* high, low
* low, high
* low, low

## 3.1 Load data for "Pitch" stimuli

In [12]:
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
inputtype = 'pitchStim'

if inputtype in ['colorStim','oriStim']:
    rois = np.where((networkdef==networkmappings['vis1']) | (networkdef==networkmappings['vis2']))[0] 
elif inputtype in ['constantStim','pitchStim']:
    rois = np.where(networkdef==networkmappings['aud'])[0]

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = tools.loadInputActivity(subj,inputtype)
    scount += 1

/projects/f_mc1689_1/AnalysisTools/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':


#### Decode different stimulus pairings

In [13]:
nproc = 30
ncvs = 1

distances_baseline_pitch = tools.conditionDecodings(data_task, rois, motorOutput=False, ncvs=ncvs, nproc=nproc)


#### Compute statistics

In [14]:
inputtype = 'pitchStim'
statistics_pitch = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = distances_baseline_pitch.shape[1]
    p = stats.binom_test(np.mean(distances_baseline_pitch[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline_pitch[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics_pitch[roicount,0] = np.mean(distances_baseline_pitch[roicount,:])
    statistics_pitch[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics_pitch[:,1])
for roicount in range(len(rois)):
    statistics_pitch[roicount,1] = qs[roicount]
    statistics_pitch[roicount,2] = h0[roicount]*statistics_pitch[roicount,0]
    
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics_pitch[:,1]<0.05)[0]
print('Number of ROIs significant for', inputtype, 'Stimuli:', sig_ind.shape[0])
print('Accuracies:', statistics_pitch[sig_ind,0])

#### Map back to surface
# Put all data into a single matrix (since we only run a single classification)
inputStim = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in rois:
    vertex_ind = np.where(glasser2==roi+1)[0]
    inputStim[vertex_ind,0] = statistics_pitch[roicount,0]
    inputStim[vertex_ind,1] = statistics_pitch[roicount,1]
    inputStim[vertex_ind,2] = statistics_pitch[roicount,2]

    roicount += 1

# np.savetxt('/projects3/SRActFlow/data/results/GroupfMRI/InputStimuliDecoding/InputStimuliRegions_PITCH2.csv', rois[np.where(statistics_pitch[:,1]<0.05)[0]], delimiter=',')
    
# #### 
# # Write file to csv and run wb_command
# outdir = '/projects3/SRActFlow/data/results/GroupfMRI/InputStimuliDecoding/'
# filename = inputtype + 'Decoding2'
# np.savetxt(outdir + filename + '.csv', inputStim,fmt='%s')
# wb_file = filename + '.dscalar.nii'
# wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
# os.system(wb_command)


Number of ROIs significant for pitchStim Stimuli: 15
Accuracies: [0.5078125  0.3046875  0.31510417 0.30729167 0.47395833 0.5234375
 0.453125   0.40885417 0.47135417 0.30729167 0.42447917 0.4375
 0.48177083 0.484375   0.41666667]


# 4.0 Run across subject decoding on Constant Stimuli decoding
### 4-way coding for every 4 input stimuli combinations
* constant, constant
* constant, beep
* beep, constant
* beep, beep

## 1.1 Load data for "Constant" stimuli

In [15]:
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
inputtype = 'constantStim'

if inputtype in ['colorStim','oriStim']:
    rois = np.where((networkdef==networkmappings['vis1']) | (networkdef==networkmappings['vis2']))[0] 
elif inputtype in ['constantStim','pitchStim']:
    rois = np.where(networkdef==networkmappings['aud'])[0]

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = tools.loadInputActivity(subj,inputtype)
    scount += 1

/projects/f_mc1689_1/AnalysisTools/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':


#### Decode different stimulus pairings

In [16]:
nproc = 30
ncvs = 1

distances_baseline_constant = tools.conditionDecodings(data_task, rois, motorOutput=False, ncvs=ncvs, nproc=nproc)


#### Compute statistics

In [17]:
inputtype = 'constantStim'
statistics_constant = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = distances_baseline_constant.shape[1]
    p = stats.binom_test(np.mean(distances_baseline_constant[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline_constant[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics_constant[roicount,0] = np.mean(distances_baseline_constant[roicount,:])
    statistics_constant[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics_constant[:,1])
for roicount in range(len(rois)):
    statistics_constant[roicount,1] = qs[roicount]
    statistics_constant[roicount,2] = h0[roicount]*statistics_constant[roicount,0]
    
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics_constant[:,1]<0.05)[0]
print('Number of ROIs significant for', inputtype, 'Stimuli:', sig_ind.shape[0])
print('Accuracies:', statistics_constant[sig_ind,0])

#### Map back to surface
# Put all data into a single matrix (since we only run a single classification)
inputStim = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in rois:
    vertex_ind = np.where(glasser2==roi+1)[0]
    inputStim[vertex_ind,0] = statistics_constant[roicount,0]
    inputStim[vertex_ind,1] = statistics_constant[roicount,1]
    inputStim[vertex_ind,2] = statistics_constant[roicount,2]

    roicount += 1

# np.savetxt('/projects3/SRActFlow/data/results/GroupfMRI/InputStimuliDecoding/InputStimuliRegions_CONSTANT2.csv', rois[np.where(statistics_constant[:,1]<0.05)[0]], delimiter=',')
    
# #### 
# # Write file to csv and run wb_command
# outdir = '/projects3/SRActFlow/data/results/GroupfMRI/InputStimuliDecoding/'
# filename = inputtype + 'Decoding2'
# np.savetxt(outdir + filename + '.csv', inputStim,fmt='%s')
# wb_file = filename + '.dscalar.nii'
# wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
# os.system(wb_command)


Number of ROIs significant for constantStim Stimuli: 12
Accuracies: [0.39583333 0.34635417 0.328125   0.3203125  0.37239583 0.3828125
 0.33854167 0.3203125  0.328125   0.34114583 0.34895833 0.359375  ]
